In [1]:
""" Problem Set 1
Part A: House Hunting

Write a program to calculate how many months it will take you to save up enough money for a down
payment. You will want your main variables to be floats, so you should cast user inputs to floats.
"""

# constants
PORTION_DOWN_PAYMENT = 0.25
R = 0.04


def calculate_months(annual_salary: float,
                     portion_saved: float,
                     total_cost: float) -> int:
    """Calculates months need to to save downpayment

    Arguments:
        annual_salary {float} -- Annual salary
        portion_saved {float} -- Portion of salary saved monthly
        total_cost {float} -- Total cost of a house

    Returns:
        int -- Months needed for downpayment

    >>> calculate_months(120000, 0.1, 1000000)
    183
    >>> calculate_months(80000, 0.15, 500000)
    105

    """

    # initial values
    current_savings = 0.0
    needed_months = 0

    # define basic rates
    monthly_salary = annual_salary / 12
    needed_down_payment = total_cost * PORTION_DOWN_PAYMENT

    # iterate over months and add up savings
    while current_savings <= needed_down_payment:
        current_savings += current_savings * R / 12
        current_savings += portion_saved * monthly_salary
        needed_months += 1

    return needed_months


# ask user for input
salary = float(input("Your starting annual salary: "))
saved = float(input("Your portion of salary to be saved: "))
cost = float(input("The cost of your dream home: "))

months = calculate_months(salary, saved, cost)
print(f"You will need {months} months to save for downpayment")


Your starting annual salary: 120000
Your portion of salary to be saved: 0.1
The cost of your dream home: 1000000
You will need 183 months to save for downpayment


In [2]:
""" Problem Set 1
Part B: Saving with a raise

Write a program to calculate how many months it will take you save up enough money for a down payment. Assume that your investments earn a return of r = 0.04 (or 4%) and the required down payment percentage is 0.25 (or 25%).
"""
# constants
PORTION_DOWN_PAYMENT = 0.25
R = 0.04


def calculate_months(annual_salary: float,
                     portion_saved: float,
                     total_cost: float,
                     semi_annual_rase: float) -> int:
    """Calculates months need to to save downpayment with semi-annual raise in salary

    Arguments:
        annual_salary {float} -- Annual salary
        portion_saved {float} -- Portion of salary saved monthly
        total_cost {float} -- Total cost of a house
        semi_annual_rase {float} -- Semi annual raise in salary

    Returns:
        float -- Months needed for downpayment

    >>> calculate_months(120000, 0.05, 500000, 0.03)
    142
    >>> calculate_months(80000, 0.1, 800000, 0.03)
    159
    >>> calculate_months(75000, 0.05, 1500000, 0.05)
    261

    """

    # initial values
    current_savings = 0.0
    month_count = 0

    # define basic rates
    needed_down_payment = total_cost * PORTION_DOWN_PAYMENT

    # iterate over months and add up savings
    while current_savings <= needed_down_payment:
        if month_count != 0 and month_count % 6 == 0:
            annual_salary += annual_salary * semi_annual_rase

        current_savings += current_savings * R / 12
        current_savings += portion_saved * annual_salary / 12
        month_count += 1

    return month_count


# ask user for input
salary = float(input("Your starting annual salary: "))
saved = float(input("Your portion of salary to be saved: "))
cost = float(input("The cost of your dream home: "))
salary_raise = float(input("Your semi annual raise in salary: "))

months = calculate_months(salary,
                            saved,
                            cost,
                            salary_raise)
print(f"You will need {months} months to save for downpayment")


Your starting annual salary: 120000
Your portion of salary to be saved: 0.05
The cost of your dream home: 500000
Your semi annual raise in salary: 0.03
You will need 142 months to save for downpayment


In [3]:
""" Problem Set 1
Part C: Finding the right amount to save away

Write a program to calculate the best savings rate, as a function of your starting salary.
You should use bisection search to help you do this efficiently.
You should keep track of the number of steps it takes your bisections search to finish.
You should be able to reuse some of the code you wrote for part B in this problem.
"""
from typing import Tuple

# constants
SEMI_ANNUAL_RAISE = 0.07
ANNUAL_RETURN = 0.04
DOWN_PAYMENT_RATE = 0.25
TOTAL_COST = 1000000
PORTION_DOWN_PAYMENT = TOTAL_COST * DOWN_PAYMENT_RATE
MONTHS = 36
BUFFER = 100
PORTION_RATE_MAX = 10000


def calculate_savings_rate(starting_salary: int) -> Tuple[float, float]:
    """Calculates the best savings rate and number of bisect steps required
    to calculate it, as a function of starting salary.

    Arguments:
        starting_salary {int} -- Starting salary

    Returns:
        Tuple[float, float] -- A tuple of rate and steps

    >>> calculate_savings_rate(150000)
    (0.4409, 11)
    >>> calculate_savings_rate(300000)
    (0.2205, 12)
    >>> calculate_savings_rate(10000)
    (0, 0)
    >>> calculate_savings_rate(1000000)
    (0.0662, 12)

    """
    rate_min = 0
    rate_max = 10000
    portion_saved = int((rate_max + rate_min) / 2)
    steps = 0
    possible_to_save = False

    while not possible_to_save:
        steps += 1

        # reset annual salary and current savings on each iteration
        annual_salary = starting_salary
        current_savings = 0.0

        # iterate over months and add up savings
        for current_month in range(1, MONTHS + 1):
            # monthly return on investment
            monthly_return = current_savings * (ANNUAL_RETURN / 12)
            # monthly savings
            monthly_savings = (annual_salary / 12) * (portion_saved / 10000)
            # total monthly savings
            current_savings = current_savings + monthly_return + monthly_savings

            # raise salary if it is time
            if current_month % 6 == 0:
                annual_salary += annual_salary * SEMI_ANNUAL_RAISE

        # if we save all money but savings cannot reach downpayment it is a failure case
        if current_savings < PORTION_DOWN_PAYMENT and portion_saved == PORTION_RATE_MAX:
            break

        # if difference between savings and downpayment is less than buffer it is a success case
        elif abs(current_savings - PORTION_DOWN_PAYMENT) <= BUFFER:
            possible_to_save = True
            break

        # none of the top cases matched, continue refining beginning or end of rate range
        if current_savings < PORTION_DOWN_PAYMENT:
            rate_min = portion_saved
        else:
            rate_max = portion_saved

        # set starting guess for next iteration
        portion_saved = (rate_min + rate_max) / 2

    if possible_to_save:
        return round(portion_saved / 10000, 4), steps
    else:
        return 0, 0


# ask user for input
salary = float(input("Your starting annual salary: "))
# calculate rate and steps
(rate, num_steps) = calculate_savings_rate(salary)
if rate == 0:
    print("It is not possible to pay the down payment in three years.")
else:
    print(f"Best savings rate (%): {rate:.4}")
    print(f"Steps in bisection search: {num_steps}")


Your starting annual salary: 300000
Best savings rate (%): 0.2205
Steps in bisection search: 12
